In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import requests
import wikipedia
import re
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from scrapy import selector
import datetime as dt
import pickle
from skimage import io
from IPython.display import clear_output
import pdb
plt.style.use('ggplot')
%matplotlib inline

# Functions

In [68]:
def get_squad(batting_sheet):
    """Gets team members, captain and keeper
    """
    team = []
    
    # batsmen who batted
    batsmen = batting_sheet.find_all('div', class_ = "wrap batsmen")
    for bat in batsmen:
        player = bat.find('div', class_ = "cell batsmen").text
        if player.find('(c)')>-1:
            captain = player.replace('(c)','').replace('†','').strip().lower()
        if player.find('†')>-1:
            keeper = player.replace('†','').replace('(c)','').strip().lower()
        player = player.replace('†','').replace('(c)','').strip()
        team.append(player.lower())
    did_not_bat = batting_sheet.find('div', class_ = "wrap dnb")
    
    
    # Now batsmen who did not bat
    for dnb in did_not_bat.find_all('a'):
        dnb = dnb.text.lower()
        if dnb.find('(c)')>-1:
            captain = dnb.replace('(c)','').replace('†','').strip().lower()
        if dnb.find('†')>-1:
            keeper = dnb.replace('†','').replace('(c)','').strip().lower()
        team.append(dnb.replace('†','').replace('(c)','').strip().lower())
    return team, captain, keeper


def get_full_name(name, team):
    """Accepts last name and returns players name
    """
    name = name.lower()
    name = name.replace('†','').replace('(c)','')
    for member in team:
        if member.lower().find(name) > -1:
            return member
        
def get_innings(innings, teams):
    """Returns batting and bowling innings
    """
    
    for i,inning in enumerate(innings):
        for team in teams:
            if inning.find(team)>-1:
                innings[i] = team
    
    return innings
                

        
def get_toss_outcome(toss_string):
    pass

In [71]:

outs = ['b', 'c', 'lbw', 'st', 'hw', 'ro', 'hand', 'obs']

def how_out(dismissal, team):
    """Returns triplet tuple as (how, fielder, bowler)
    """
    
    dismissal = dismissal.lower().strip()
    print(dismissal)
    
    if dismissal == 'not out':
        return 'not out', '', ''
    
    if dismissal == 'retired hurt':
        return 'retired hurt', '', ''
    
    if dismissal.find('absent') > -1 and dismissal.find('hurt') > -1:
        return 'absent hurt', '', ''
    
    
    
    ## No fielder involved
    # run out
    if dismissal.find('run out')>-1:
        how = 'run out'
        fielder = ''
        bowler = ''
        return how, fielder, bowler
    
    # handled
    if dismissal.find('handled')>-1:
        how = 'handled the ball'
        fielder = ''
        bowler = ''
        return how, fielder, bowler

    # obstructing the fieldsman
    if dismissal.find('obstruct') > -1:
        how = 'obstruct'
        fielder = ''
        blower = ''
        return how, fielder, bowler
    
    bowler = re.findall(r"b\s(.+)", dismissal)
    fielder = re.findall(r"c\s(.+)\sb", dismissal)
    
    if len(bowler)>0:
        bowler = bowler[0]
    if len(fielder)>0:
        fielder = fielder[0]
    
    #print(bowler, fielder)
    
    if len(fielder) == 0:
        keeper = re.findall(r"st\s†(.+)\sb", dismissal)
        if len(keeper)>0:
            keeper = keeper[0]
    
    # bowled
    if len(fielder) == 0 and dismissal == 'b ' + bowler:
        how = 'bowled'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # caught
    if len(fielder) > 0:
        how = 'caught'
        fielder = get_full_name(fielder, team)
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # lbw
    if dismissal.find('lbw')>-1 or dismissal.find('leg before')>-1:
        how = 'lbw'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    # hit wicket
    if dismissal.find('hit wicket')>-1:
        how = 'hit wicket'
        fielder = ''
        bowler = get_full_name(bowler, team)
        return how, fielder, bowler
    
    
    # caught and bowled
    if dismissal.find('&') > -1:
        how = 'caught'
        bowler = get_full_name(bowler, team)
        fielder = bowler
        return how, fielder, bowler
    
    
    # Stumped
    if dismissal.find('st †') > -1:
        how = 'stumped'
        fielder = get_full_name(keeper, team)
        bowler = get_full_name(bowler, team)
        return 'stumped',fielder, bowler


In [64]:
# how_out('st †Kaluwitharana b Muralitharan', squad['sri lanka'])

In [65]:
match[1996]

,test_index,team_1,team_2,winner,margin,ground,match_date,scorecard_link
0,test # 1321,south africa,england,south africa,10 wickets,cape town,"jan 2-4, 1996",http://stats.espncricinfo.com//ci/engine/match...
1,test # 1322,new zealand,zimbabwe,drawn,,hamilton,"jan 13-17, 1996",http://stats.espncricinfo.com//ci/engine/match...
2,test # 1323,new zealand,zimbabwe,drawn,,auckland,"jan 20-24, 1996",http://stats.espncricinfo.com//ci/engine/match...
3,test # 1324,australia,sri lanka,australia,148 runs,adelaide,"jan 25-29, 1996",http://stats.espncricinfo.com//ci/engine/match...
4,test # 1325,west indies,new zealand,west indies,10 wickets,bridgetown,"apr 19-23, 1996",http://stats.espncricinfo.com//ci/engine/match...
5,test # 1326,west indies,new zealand,drawn,,st john's,"apr 27-may 2, 1996",http://stats.espncricinfo.com//ci/engine/match...
6,test # 1327,england,india,england,8 wickets,birmingham,"jun 6-9, 1996",http://stats.espncricinfo.com//ci/engine/match...
7,test # 1328,england,india,drawn,,lord's,"jun 20-24, 1996",http://stats.espncricinfo.com//ci/engine/match...
8,test # 1329,england,india,drawn,,nottingham,"jul 4-9, 1996",http://stats.espncricinfo.com//ci/engine/match...
9,test # 1330,england,pakistan,pakistan,164 runs,lord's,"jul 25-29, 1996",http://stats.espncricinfo.com//ci/engine/match...


In [75]:
%%time
def get_scoresheet(match, year, match_format):
#     match = pickle.load(open('test_list', "rb" ))
#     year = 1996
#     num = 19

    # Initialize dictionary to be returned
    all_info = {}
    
    # Reset index
    match.reset_index(inplace=True)
    index_name = match.columns[0]

    # Get match metadata
    for row in match.iterrows():
    
        link = row[1].scorecard_link
        clear_output()
        matchID = row[1][index_name]
        print(link)
        print(matchID)

        if matchID.find('odi')>-1:
            match_format = 'odi'
        elif matchID.find('test')>-1:
            match_format = 'test'

        team_1 = row[1].team_1
        team_2 = row[1].team_2
        winner = row[1].winner
        team_names = [team_1, team_2]
        opponents = {}
        opponents[team_1] = team_2
        opponents[team_2] = team_1

        # Send get request and get HTML
        r = requests.get(link)
        soup = BeautifulSoup(r.content,'lxml')

        # Number of score sheets = number of innings
        # Each scoresheet contains both batting and bowling information
        scoresheets = soup.find_all('article', class_ = "sub-module scorecard")
        match_details = soup.find('div', class_ = "match-detail-container")

        ##############################################################################################
        ############################# .   Get MATCH DETAILS   ############################################
        ##############################################################################################


        MATCH_DETAILS = {}
        MATCH_DETAILS['stadium'] = match_details.find('div', class_ = "stadium-details").text.lower().strip()
        for detail in match_details.find_all('div', class_ = "match-detail--item"):
            left = detail.find('div', class_ = "match-detail--left").text.lower().strip()
            MATCH_DETAILS[left] = []
            for r in detail.find('div', class_ = "match-detail--right").find_all('span'):
                MATCH_DETAILS[left].append(r.text.lower().strip())
        MATCH_DETAILS
        toss = re.findall(r'(.+),',MATCH_DETAILS['toss'][0])[0].strip()
        stadium = MATCH_DETAILS['stadium']


        ##############################################################################################
        ############################      Get TEAM INFO    ###########################################
        ##############################################################################################


        num_innings = len(scoresheets)

        bat_innings = []
        ball_innings = []

        squad = {}
        squad[team_1] = []
        squad[team_2] = []

        captain = {}
        captain[team_1] = []
        captain[team_2] = []

        keeper = {}
        keeper[team_1] = []
        keeper[team_2] = []

        # Get innings Info, team, captain, keeper
        for i,sheet in enumerate(scoresheets):


            if match_format == 'odi':
                my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
                this_innings = re.findall(r'(.+)\sinnings', my_str)[0]
            elif match_format == 'test':
                my_str = sheet.find('div', class_ = "accordion-header").text.lower().strip()
                this_innings = re.findall(r'(.+)\s\d', my_str)[0]

            bat_innings.append(this_innings)

        ball_innings = [opponents[x] for x in bat_innings]

        # Get squad for team batting first
        team_batting_first = bat_innings[0]
        batting_sheet = scoresheets[0].find("div", class_="scorecard-section batsmen")
        squad[team_batting_first], captain[team_batting_first], keeper[team_batting_first] = get_squad(batting_sheet)

        # Get squad for team batting second
        team_batting_second = bat_innings[1]
        batting_sheet = scoresheets[1].find("div", class_="scorecard-section batsmen")
        squad[team_batting_second], captain[team_batting_second], keeper[team_batting_second] = get_squad(batting_sheet)


        TEAM_INFO = {}
        TEAM_INFO['num_innings'] = num_innings

        TEAM_INFO['bat_innings'] = bat_innings
        TEAM_INFO['ball_innings'] = ball_innings

        TEAM_INFO['team_batting_first'] = team_batting_first
        TEAM_INFO['team_batting_second'] = team_batting_second

        TEAM_INFO['squad'] = squad
        TEAM_INFO['captain'] = captain
        TEAM_INFO['keeper'] = keeper




        #############################################################################################
        ####################    Get Scoresheets/ Batting and Bowling        #########################
        #############################################################################################


        # Now get batting and bowling info
        BATTING = {}
        BOWLING = {}
        for s,sheet in enumerate(scoresheets):

            # batting_sheet = sheet.find("div", class_="scorecard-section batsmen")
            # bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
            inning = 'inning_' + str(s+1)
            print("In:", inning)

            batting_team = bat_innings[s]
            bowling_team = ball_innings[s]

            #####################################################
            ############### Batting Info  #######################
            #####################################################


            batting_sheet = sheet.find("div", class_="scorecard-section batsmen")

            ## DEFINE HEADERS
            # get all the header.
            # Some games have SR and balls, some not, so important to get it on a game-by-game basis
            all_headers = []
            for header in batting_sheet.find('div', class_ = "wrap header").find_all('div'):
                all_headers.append(header.text.lower())

            # The commentary header is usually emptty: just add commentary
            if len(all_headers[1])==0:
                all_headers[1] = 'commentary'

            # Create dataframe
            all_headers =  all_headers + ['how', 'fielder', 'bowler'] + ['matchID', 'year', 'team', 'innings', 'toss', 'result', 'stadium']
            score_card_batting = pd.DataFrame(columns = all_headers)

            # Iterate over batesmen
            all_batsmen  = batting_sheet.find_all('div', class_ = "wrap batsmen")
            for i, batsman in enumerate(all_batsmen):

                # iterate over cells
                for j,info in enumerate(batsman.find_all('div')):
                    # print(j,info.text, type(info.text))

                    if j == 0: #name
                        player = info.text.lower().replace('(c)','').replace('†','').strip()
                        score_card_batting.loc[i, all_headers[j]] = player
                        continue
                    if j == 1: #commentary
                        # print(info.text.lower().strip())
                        how,fielder,bowler = how_out(info.text.lower().strip(), squad[bowling_team])


                    score_card_batting.loc[i, 'how'] = how
                    score_card_batting.loc[i, 'fielder'] = fielder
                    score_card_batting.loc[i, 'bowler'] = bowler
                    score_card_batting.loc[i, all_headers[j]] = info.text.lower().strip()

            did_not_bat = set(squad[batting_team]) - set(score_card_batting.batsmen.values)
            #print(did_not_bat)

            l = len(score_card_batting)
            for dnb in did_not_bat:
                score_card_batting.loc[l, all_headers[0]] = dnb
                l = l+1

            # Now get extras
            extras = sheet.find('div', class_ = "wrap extras").find_all('div')[1].text
            l = len(score_card_batting)
            score_card_batting.loc[l, all_headers[0]] = 'extras'
            score_card_batting.loc[l, all_headers[1]] =  extras

            # Total
            total = sheet.find('div', class_ = "wrap total").find_all('div')[1].text
            l = len(score_card_batting)
            score_card_batting.loc[l, all_headers[0]] = 'total'
            score_card_batting.loc[l, all_headers[1]] =  total


            # Now fill match-cells: common to all batsmen
            score_card_batting.loc[:, 'matchID'] = matchID
            score_card_batting.loc[:, 'year'] = year
            score_card_batting.loc[:, 'innings'] = s+1
            score_card_batting.loc[:, 'team'] = batting_team

            score_card_batting.loc[:, 'stadium'] = stadium
            score_card_batting.loc[:, 'toss'] = toss
            score_card_batting.loc[:, 'result'] = winner

            BATTING[inning] = score_card_batting.fillna(0)

            #####################################################
            ############### Bowling Info  #######################
            #####################################################


            bowling_sheet = sheet.find("div", class_ = "scorecard-section bowling")
            bowling_headers = []
            for header in bowling_sheet.find('thead').find_all('th'):
                bowling_headers.append(header.text.lower().strip())

            bowling_headers = bowling_headers +  ['matchID', 'year', 'team', 'innings']
            score_card_bowling = pd.DataFrame(columns = bowling_headers)  

            # bowlers who bowled
            for b,bowler in enumerate(bowling_sheet.find('tbody').find_all('tr')): 
                for c,cell in enumerate(bowler.find_all('td')):
                    score_card_bowling.loc[b,bowling_headers[c]]= cell.text.lower().strip()
            # bowlers who did not bowl
            dnb = set(squad[bowling_team]) - set(score_card_bowling[bowling_headers[0]].values)
            l = len(score_card_bowling)
            for d in dnb:
                score_card_bowling.loc[l,bowling_headers[0]] = d
                l = l+1

            # Now fill match-cells: common to all bowlers
            score_card_bowling.loc[:, 'matchID'] = matchID
            score_card_bowling.loc[:, 'year'] = year
            score_card_bowling.loc[:, 'innings'] = s+1
            score_card_bowling.loc[:, 'team'] = bowling_team

            score_card_bowling.loc[:, 'stadium'] = stadium
            score_card_bowling.loc[:, 'toss'] = toss
            score_card_bowling.loc[:, 'result'] = winner

            BOWLING[inning] = score_card_bowling.fillna(0)



        ## SAVE EVERYTHING IN 1 DICTIONARY
        all_info[matchID] = {}
        all_info[matchID]['team_info'] = TEAM_INFO
        all_info[matchID]['match_details'] = MATCH_DETAILS
        all_info[matchID]['batting'] = BATTING
        all_info[matchID]['bowling'] = BOWLING
    
    return all_info

CPU times: user 29 µs, sys: 1 µs, total: 30 µs
Wall time: 34.8 µs


In [76]:
tests = pickle.load(open('test_list', "rb" ))
year = 1960
num = 19
match_format = 'test'
info = get_scoresheet(tests[year], year, match_format)

http://stats.espncricinfo.com//ci/engine/match/62885.html
test # 501
In: inning_1
c baig b desai
b surendranath
c nadkarni b surendranath
lbw b borde
c umrigar b desai
c jaisimha b borde
b surendranath
c †tamhane b surendranath
lbw b borde
b borde
not out
In: inning_2
b intikhab alam
c mathias b mahmood hussain
b intikhab alam
c †imtiaz ahmed b mahmood hussain
b fazal mahmood
c †imtiaz ahmed b fazal mahmood
c †imtiaz ahmed b fazal mahmood
b haseeb ahsan
c intikhab alam b fazal mahmood
not out
b fazal mahmood
In: inning_3
not out
b desai
lbw b surendranath
run out
not out
In: inning_4
c fazal mahmood b haseeb ahsan
c mathias b intikhab alam
b haseeb ahsan
not out
b intikhab alam
not out


In [81]:
info['test # 484']['batting']['inning_1']

,batsmen,commentary,r,4s,6s,,how,fielder,bowler,matchID,year,team,innings,toss,result,stadium
0,p roy,b davidson,6,-,0,,bowled,,ak davidson,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
1,nj contractor,c benaud b meckiff,108,-,0,,caught,r benaud,i meckiff,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
2,pr umrigar,c harvey b davidson,0,0,0,,caught,rn harvey,ak davidson,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
3,aa baig,c †grout b davidson,50,-,0,,caught,atw grout,ak davidson,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
4,cg borde,b meckiff,26,-,0,,bowled,,i meckiff,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
5,gs ramchand,lbw b meckiff,0,0,0,,lbw,,i meckiff,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
6,rb kenny,b meckiff,20,-,0,,bowled,,i meckiff,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
7,bk kunderan,lbw b lindwall,19,-,0,,lbw,,rr lindwall,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
8,rg nadkarni,not out,18,-,0,,not out,,,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"
9,sa durani,c stevens b benaud,18,-,0,,caught,0,r benaud,test # 484,1960,india,1,india,drawn,"brabourne stadium, mumbai"


In [41]:
TEAM_INFO

{'num_innings': 4,
 'bat_innings': ['australia', 'west indies', 'australia', 'west indies'],
 'ball_innings': ['west indies', 'australia', 'west indies', 'australia'],
 'team_batting_first': 'australia',
 'team_batting_second': 'west indies',
 'squad': {'australia': ['ma taylor',
   'mtg elliott',
   'rt ponting',
   'me waugh',
   'sr waugh',
   'mg bevan',
   'ia healy',
   'pr reiffel',
   'sk warne',
   'ms kasprowicz',
   'gd mcgrath'],
  'west indies': ['sl campbell',
   'rg samuels',
   'bc lara',
   'cl hooper',
   's chanderpaul',
   'jc adams',
   'co browne',
   'ir bishop',
   'cel ambrose',
   'kcg benjamin',
   'ca walsh']},
 'captain': {'australia': 'ma taylor', 'west indies': 'ca walsh'},
 'keeper': {'australia': 'ia healy', 'west indies': 'co browne'}}

In [43]:
BOWLING['inning_1']

,bowling,,o,m,r,w,econ,wd,nb,,matchID,year,team,innings,stadium,toss,result
0,cel ambrose,,34,4,93,1,2.73,1,7,,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
1,ca walsh,,35,6,112,4,3.20,0,10,,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
2,kcg benjamin,,33,6,97,2,2.93,2,5,,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
3,ir bishop,,30,2,105,3,3.50,0,5,,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
4,cl hooper,,19,3,64,0,3.36,0,0,,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
5,rg samuels,0,0,0,0,0,0,0,0,0,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
6,co browne,0,0,0,0,0,0,0,0,0,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
7,jc adams,0,0,0,0,0,0,0,0,0,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
8,sl campbell,0,0,0,0,0,0,0,0,0,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia
9,bc lara,0,0,0,0,0,0,0,0,0,test # 1340,1996,west indies,1,"brisbane cricket ground, woolloongabba, brisbane",west indies,australia


In [45]:
BATTING['inning_2']

,batsmen,commentary,r,b,m,4s,6s,sr,,how,fielder,bowler,matchID,year,team,innings,toss,result,stadium
0,sl campbell,c warne b reiffel,18,58,80,2,0,31.03,,caught,sk warne,pr reiffel,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
1,rg samuels,c †healy b mcgrath,10,37,47,2,0,27.02,,caught,ia healy,gd mcgrath,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
2,bc lara,c me waugh b mcgrath,26,69,95,3,0,37.68,,caught,me waugh,gd mcgrath,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
3,cl hooper,c ponting b sr waugh,102,228,306,10,1,44.73,,caught,rt ponting,sr waugh,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
4,s chanderpaul,c me waugh b reiffel,82,230,284,4,0,35.65,,caught,me waugh,pr reiffel,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
5,jc adams,lbw b ponting,0,6,10,0,0,0.00,,lbw,,rt ponting,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
6,co browne,c †healy b reiffel,4,27,40,0,0,14.81,,caught,ia healy,pr reiffel,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
7,ir bishop,lbw b warne,0,6,6,0,0,0.00,,lbw,,sk warne,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
8,cel ambrose,c sub (jn gillespie) b reiffel,0,1,11,0,0,0.00,,caught,0,0,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"
9,kcg benjamin,lbw b warne,9,7,4,2,0,128.57,,lbw,,sk warne,test # 1340,1996,west indies,2,west indies,australia,"brisbane cricket ground, woolloongabba, brisbane"


In [46]:
MATCH_DETAILS

{'stadium': 'brisbane cricket ground, woolloongabba, brisbane',
 'series': ['west indies tour of australia 1996/97'],
 'toss': ['west indies , elected to field first'],
 'player of the match': ['ian healy'],
 'series result': ['australia led the 5-match series 1-0'],
 'match number': ['test no. 1340'],
 'season': ['1996/97'],
 'match days': ['22,23,24,25,26 november 1996 (5-day match)'],
 'test debut': ['matthew elliott', 'michael kasprowicz'],
 'umpires': ['steve randell', 'cyril mitchley'],
 'tv umpires': ['peter parker'],
 'match referee': ['peter van der merwe']}

In [47]:
BATTING['inning_1'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 19 columns):
batsmen       13 non-null object
commentary    13 non-null object
r             13 non-null object
b             13 non-null object
m             13 non-null object
4s            13 non-null object
6s            13 non-null object
sr            13 non-null object
              13 non-null object
how           13 non-null object
fielder       13 non-null object
bowler        13 non-null object
matchID       13 non-null object
year          13 non-null int64
team          13 non-null object
innings       13 non-null int64
toss          13 non-null object
result        13 non-null object
stadium       13 non-null object
dtypes: int64(2), object(17)
memory usage: 2.7+ KB
